In [117]:
import pandas as pd
import numpy as np
from itertools import chain

In [95]:
df = pd.read_csv('prepost_matched_debt_files.csv', index_col = 0)

In [96]:
def stringToList(x):
    return x.strip('][\"\'').split(', ') if not pd.isnull(x) else x

In [97]:
df['full name'] = df['full name'].apply(lambda x: stringToList(x))
df['Same State Loan Office'] = df['Same State Loan Office'].apply(lambda x: 
                                                                  stringToList(x))
df['Different State Loan Office'] = df['Different State Loan Office'].apply(lambda x: 
                                                                            stringToList(x))
df['Same State Liquidated Debt'] = df['Same State Liquidated Debt'].apply(lambda x: 
                                                                          stringToList(x))
df['Different State Liquidated Debt'] = df['Different State Liquidated Debt'].apply(lambda x: 
                                                                                    stringToList(x))
df['Pierce Certificates'] = df['Pierce Certificates'].apply(lambda x: 
                                                            stringToList(x))

In [205]:
df

,full name,state,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
0,[Bernard O'Neill],MD,[Bernard O'Neill],NaN,NaN,NaN,NaN,230.31,118.85973,0.0000,0.0,0.0,0.0,118.85973,1
1,[Francis O'Neill],MD,[Francis O'Neill],[Francis O Neill],NaN,NaN,NaN,37.96,17.50000,151.5125,0.0,0.0,0.0,169.01250,2
2,[Henry O'Neale],MD,NaN,NaN,NaN,NaN,[Henry O'Neal],99.77,0.00000,0.0000,0.0,0.0,23.0,23.00000,1
3,[James O'Hara],PA,NaN,NaN,NaN,NaN,NaN,33.44,0.00000,0.0000,0.0,0.0,0.0,0.00000,0
4,[John D'Mercier],NY,NaN,NaN,NaN,NaN,NaN,0.00,0.00000,0.0000,0.0,0.0,0.0,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4947,[Zephaniah Andrews],RI,NaN,NaN,NaN,NaN,NaN,1728.74,0.00000,0.0000,0.0,0.0,0.0,0.00000,0
4948,[Zephaniah Brown],RI,NaN,NaN,NaN,NaN,NaN,2415.08,0.00000,0.0000,0.0,0.0,0.0,0.00000,0
4949,[Zephaniah Davis],CT,NaN,NaN,NaN,NaN,NaN,77.20,0.00000,0.0000,0.0,0.0,0.0,0.00000,0
4950,[Zerujah Bearley],NJ,NaN,NaN,NaN,NaN,NaN,0.00,0.00000,0.0000,0.0,0.0,0.0,0.00000,0


# How many unique individuals were issued 6 percent stocks and deferred 6 percent stocks in 1790 and after?

In [98]:
df.groupby('state')['full name'].count()

state
CT    747
GA     54
MD    356
NC     54
NH    169
NJ    566
NY    882
PA    839
RI    502
SC    274
VA    509
Name: full name, dtype: int64

# How many of these individuals
- were original purchasers of loan office certicates of the same state as the 6 percent stock?
- were original purchasers of loan office certicates issued from another state?
- were original recipients of liquidated debt certicates issued by the same-state loan office? other state loan offices?
- were original recipients of the Pierce Certicates?

### How many individuals were original purchasers of loan office certicates of the same state as the 6 percent stock?

In [99]:
df[df['Same State Loan Office'].apply(lambda x: 
                                      type(x) == list)]['state'].value_counts()

CT    327
PA    307
MD    102
VA     82
NH     63
NY     56
NJ     24
RI     15
SC     14
GA      2
Name: state, dtype: int64

### How many individuals were original purchasers of loan office certicates of a different state as the 6 percent stock?

In [100]:
df[df['Different State Loan Office'].apply(lambda x: 
                                           type(x) == list)]['state'].value_counts()

PA    128
CT    115
VA    103
MD     59
NY     44
NH     42
NJ     30
SC     18
RI     16
GA      2
NC      1
Name: state, dtype: int64

##### Aside: How many individuals were original purchasers of loan office certicates of a different and same state as the 6 percent stock?

In [101]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Loan Office'], 
                  df['Different State Loan Office'])]]['state'].value_counts()

PA    84
CT    67
VA    39
MD    21
NJ    16
NH    14
NY    14
SC    10
RI     8
GA     2
Name: state, dtype: int64

### How many individuals were original recipients of liquidated debt certificates issued by the same state loan office?

In [102]:
df[df['Same State Liquidated Debt'].apply(lambda x: 
                                          type(x) == list)]['state'].value_counts()

PA    213
CT    107
NY     51
NH     29
VA     14
NJ      9
SC      7
RI      6
MD      3
GA      1
Name: state, dtype: int64

### How many individuals were original recipients of liquidated debt certificates issued by another state loan office?

In [103]:
df[df['Different State Liquidated Debt'].apply(lambda x: 
                                               type(x) == list)]['state'].value_counts()

CT    185
PA    131
NY     74
NH     59
VA     59
MD     41
NJ     34
RI     21
SC     15
GA      1
Name: state, dtype: int64

##### Aside: How many individuals were original purchasers of liquidated debt certificates from the same and another loan office?

In [104]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Liquidated Debt'], 
                  df['Different State Liquidated Debt'])]]['state'].value_counts()

CT    107
PA     65
NY     48
NH     29
VA     13
NJ      8
RI      6
SC      5
MD      3
GA      1
Name: state, dtype: int64

### How many individuals were original recipients of the Pierce Certicates, corresponding to the same state?

In [105]:
df[df['Pierce Certificates'].apply(lambda x: 
                                   type(x) == list)]['state'].value_counts()

CT    182
PA    166
VA    146
MD     82
NY     55
NH     35
NJ     23
SC     16
RI     15
NC      2
GA      1
Name: state, dtype: int64

### For how many individuals do the face values of their pre-1790 certificates equal the sum of the face values of the 6 percent and deferred 6 percent stocks that they received?

##### Number of people with non-zero assets and that the debt values from pre-1790 and post-1790 are equal

In [208]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]

,full name,state,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
16,[Aaron Kelsey],CT,[Aaron Kelsey],NaN,NaN,NaN,NaN,95.10,95.10834,0.0,0.000000,0.0,0.0,95.108340,1
29,[Abel Morse],CT,[Abel Morse],NaN,NaN,NaN,NaN,1388.83,1388.86930,0.0,0.000000,0.0,0.0,1388.869300,1
30,[Abel Patchen],CT,[Abel Patchen],NaN,NaN,NaN,NaN,153.96,153.97916,0.0,0.000000,0.0,0.0,153.979160,1
58,[Abner Corson],PA,[Abner Corson],NaN,NaN,NaN,NaN,60.23,60.23889,0.0,0.000000,0.0,0.0,60.238890,1
67,[Abraham Brinsmade],CT,[Abraham Brinsmade],NaN,NaN,NaN,NaN,400.00,400.00000,0.0,0.000000,0.0,0.0,400.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686,[William Dickensheet],MD,[William Dickensheet],NaN,NaN,NaN,NaN,50.00,50.00833,0.0,0.000000,0.0,0.0,50.008330,1
4757,[William House],MD,[William House],NaN,NaN,NaN,NaN,25.47,25.48750,0.0,0.000000,0.0,0.0,25.487500,1
4863,[William Sitgreaves],PA,NaN,NaN,[William Sitgreaves],NaN,NaN,1277.57,0.00000,0.0,1277.577778,0.0,0.0,1277.577778,1
4929,[William Wyvell],MD,[William Wyvell],NaN,NaN,NaN,NaN,22.50,22.50000,0.0,0.000000,0.0,0.0,22.500000,1


In [211]:
df[df['tot_pre1790_certs'] != 0].shape[0]

1703

In [106]:
df[[tt == dt and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

14

##### State Distribution

In [210]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

CT    50
MD    18
PA    16
NH     6
VA     5
Name: state, dtype: int64

##### Number of people with zero assets and equal values
Most of these occur when someone has 3% but no 6% certificates

In [108]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

1299

##### State Distribution

In [109]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

NY    688
NJ    525
VA     26
CT     23
RI     17
PA      9
MD      8
SC      3
Name: state, dtype: int64

### Who, and how many people redeemed certificates with Hamilton and were not original owners/purchasers

In [227]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]].shape[0]

1972

In [218]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]

,full name,state,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
3,[James O'Hara],PA,NaN,NaN,NaN,NaN,NaN,33.44,0.0,0.0,0.0,0.0,0.0,0.0,0
6,[A McWhoker],NY,NaN,NaN,NaN,NaN,NaN,340.70,0.0,0.0,0.0,0.0,0.0,0.0,0
8,[Aaron Bourn],RI,NaN,NaN,NaN,NaN,NaN,309.55,0.0,0.0,0.0,0.0,0.0,0.0,0
10,[Aaron Cadwell II],CT,NaN,NaN,NaN,NaN,NaN,13.88,0.0,0.0,0.0,0.0,0.0,0.0,0
11,[Aaron Colvin],RI,NaN,NaN,NaN,NaN,NaN,79.05,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4945,[Zebulon Waterman],CT,NaN,NaN,NaN,NaN,NaN,306.61,0.0,0.0,0.0,0.0,0.0,0.0,0
4947,[Zephaniah Andrews],RI,NaN,NaN,NaN,NaN,NaN,1728.74,0.0,0.0,0.0,0.0,0.0,0.0,0
4948,[Zephaniah Brown],RI,NaN,NaN,NaN,NaN,NaN,2415.08,0.0,0.0,0.0,0.0,0.0,0.0,0
4949,[Zephaniah Davis],CT,NaN,NaN,NaN,NaN,NaN,77.20,0.0,0.0,0.0,0.0,0.0,0.0,0


##### state distribution

In [219]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['state'].value_counts()

RI    456
PA    348
VA    260
SC    246
CT    246
MD    172
NY     86
NH     55
GA     52
NC     51
Name: state, dtype: int64

In [220]:
namelsts = df[[tot == 0 and tt != 0 for 
               tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x)).values
names = [x for x in list(chain.from_iterable(namelsts)) if not pd.isnull(x)]

In [221]:
name_state_asset = pd.DataFrame({}, columns = ['Full Name', 'Total', 'state'])

In [222]:
name_df = df[[tot == 0 and tt != 0 for 
              tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x))
for name in names:
    ind1 = name_df[name_df[0] == name].index.tolist()
    ind2 = name_df[name_df[1] == name].index.tolist()
    ind3 = name_df[name_df[2] == name].index.tolist()
    ind1.extend(ind2)
    ind1.extend(ind3)
    ind1 = list(set(ind1))
    tot_assets = df.loc[ind1, 'Total'].sum()
    state = df.loc[ind1, 'state'].values[0]
    add_df = pd.DataFrame()
    add_df['Full Name'] = [name]
    add_df['Total'] = [tot_assets]
    add_df['state'] = [state]
    name_state_asset = pd.concat([name_state_asset, add_df])

In [223]:
cd_names = pd.DataFrame(name_state_asset.groupby(['Full Name', 'state'])['Total'].sum()).reset_index()

In [224]:
cd_names.to_csv("speculator_names.csv")